<a href="https://colab.research.google.com/github/znumunz/dataviz2025/blob/main/Copy_of_Test_Final_DataViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# นำเข้าข้อมูล

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
# base_path = '/content/drive/MyDrive/Data_Final_DataViz'

In [ ]:
import pandas as pd

In [ ]:
df_nf_film = pd.read_excel("/content/drive/MyDrive/Final DataViz2025/Netflix-movies.xlsx",sheet_name='Netflix Films')
df_nf_scores = pd.read_excel("/content/drive/MyDrive/Final DataViz2025/Netflix-movies.xlsx",sheet_name='Rotten Tomatoes Score')
df_im = pd.read_csv("/content/drive/MyDrive/Final DataViz2025/imdb_top_1000.csv")

# เช็คข้อมูล

In [ ]:
df_nf_film.head(3)

In [ ]:
df_nf_film.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
df_nf_film['Release Year'].plot(kind='hist', bins=30)
plt.title('Distribution of Movie Release Years')
plt.xlabel('Release Year')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
df_nf_scores.head(3)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
df_nf_scores['TOMATOMETER'].plot(kind='hist', bins=30)
plt.title('Distribution of TOMATOMETER Scores')
plt.xlabel('TOMATOMETER Score')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
df_nf_scores.isnull().sum()

In [ ]:
df_im.head(3)

In [ ]:
df_im.isnull().sum()

#ปรับค่าคอลัมน์เป็นตัวพิพม์เล็กและลบค่าว่างกับตัวอักษรพิเศษ

In [ ]:
# Function to clean movie titles
def clean_title(title):
    if isinstance(title, str):
        title = title.lower().strip()
        # Remove special characters (keeping letters, numbers, and spaces)
        title = ''.join(e for e in title if e.isalnum() or e.isspace())
    return title

# Apply cleaning to df_nf_film before merging
df_nf_film['Movie Title'] = df_nf_film['Movie Title'].apply(clean_title)
df_nf_scores['Movie Title'] = df_nf_scores['Movie Title'].apply(clean_title)
df_im['Series_Title'] = df_im['Series_Title'].apply(clean_title)

#Merge netflix_film and tomato_scores

In [ ]:

df = pd.merge(df_nf_film, df_nf_scores, on='Movie Title', how='inner')
df

In [ ]:
df.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

# Calculate the number of null values per column
null_counts = df.isnull().sum()

# Filter out columns with no null values for better visualization
null_counts = null_counts[null_counts > 0]

if not null_counts.empty:
    plt.figure(figsize=(10, 6))
    null_counts.plot(kind='bar')
    plt.title('Number of Null Values per Column in df')
    plt.xlabel('Column')
    plt.ylabel('Number of Null Values')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("No null values found in the DataFrame df.")

#Merge (netflix_film and tomato_scores) and IMDB



In [ ]:

df1 = pd.merge(df, df_im, left_on='Movie Title',right_on ='Series_Title', how='left')
# df1['Series_Title'].isnull().sum()
df1

# กำหนด tpye

In [ ]:
df1['Type'] = df1['IMDB_Rating'].apply(lambda x: 'Movie' if pd.isnull(x) else 'Original Netflix Films')
display(df1.head())

#เลือกเฉพราะคอลัมน์ที่ต้องการ

In [ ]:
df_1=df1[['Movie Title','Director_x','Genre_x','Country','Rating','TOMATOMETER','AUDIENCE SCORE','IMDB_Rating','Type','Release Year']]
df_1

In [ ]:
df1 = df1.rename(columns=lambda x: x.replace('_x', '') if '_x' in x else x)
display(df1)

#fillna "IMDB_Rating":0

In [ ]:
df_1 = df_1.fillna({"IMDB_Rating":0})
df_1

In [ ]:
df_1=df_1.dropna()

In [ ]:
df_1.isnull().sum()

In [ ]:
df_1

In [ ]:
import matplotlib.pyplot as plt

# Count the occurrences of each rating
rating_counts = df_1['Type'].value_counts()

# Create a pie chart
plt.figure(figsize=(4, 4)) # Increase figure size further
plt.pie(rating_counts, labels=rating_counts.index, autopct='%1.1f%%', startangle=140, textprops={'fontsize': 10})
plt.title('Distribution of Movie Ratings')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
df_1['AUDIENCE SCORE'].plot(kind='hist', bins=20)
plt.title('Distribution of Audience Scores')
plt.xlabel('Audience Score')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

average_scores_by_year.plot(kind='bar', figsize=(12, 7))
plt.title('Average TOMATOMETER and Audience Score by Release Year (Last 10 Years)')
plt.xlabel('Release Year')
plt.ylabel('Score')
plt.xticks(rotation=45, ha='right')
plt.legend(['TOMATOMETER', 'AUDIENCE SCORE'])
plt.tight_layout()
plt.show()

#normalize data แยก director และ genre ออก ไมให้หลายค่าอยู่ในคอลัมน์เดียวกัน

In [ ]:
import pandas as pd
import numpy as np
import sys

# --- ส่วนที่ 1:  ---

def clean_title(title):
    """
    ฟังก์ชันทำความสะอาดชื่อหนัง
    - แปลงเป็นตัวพิมพ์เล็ก ลบขอบ
    - ลบอักขระพิเศษ
    - * สำคัญ: ถ้า input ไม่ใช่ string (เช่น ตัวเลข) จะคืนค่า NaN *
    """
    if isinstance(title, str):
        title = title.lower()
        title = title.strip()
        # Remove special characters (keeping letters, numbers, and spaces)
        title = ''.join(e for e in title if e.isalnum() or e.isspace())

        # ถ้าผลลัพธ์เป็นสตริงว่างเปล่า ก็ให้เป็น NaN
        if not title:
            return np.nan
        return title

    # ถ้าไม่ใช่ string (เช่น ตัวเลขจากไฟล์ Excel) ให้คืนค่า NaN
    return np.nan

def normalize_column(df, id_col, value_col):
    """
    ฟังก์ชันสำหรับแตกคอลัมน์ที่มีหลายค่า (เช่น "Action, Drama") ออกเป็นหลายแถว

    Parameters:
    - df (DataFrame): ตารางข้อมูล
    - id_col (str): ชื่อคอลัมน์ที่เป็น ID หลัก (เช่น 'Movie Title Cleaned')
    - value_col (str): ชื่อคอลัมน์ที่ต้องการแตก (เช่น 'Genre' หรือ 'Director')
    """

    # 1. เลือกเฉพาะคอลัมน์ที่ต้องการ และลบแถวที่มีค่าว่าง (NaN) ในคอลัมน์ใดคอลัมน์หนึ่ง
    df_normalized = df[[id_col, value_col]].dropna(subset=[id_col, value_col])

    # 2. แปลงคอลัมน์ค่า (เช่น Genre, Director) เป็น string เพื่อให้ .str.split ทำงานได้
    df_normalized = df_normalized.astype({value_col: str})

    # 3. สร้าง List โดยการ split ค่าด้วย ',' (เช่น "A, B" -> ["A", "B"])
    # .assign จะสร้างคอลัมน์ใหม่ชั่วคราวชื่อ 'SplitVal'
    df_normalized = df_normalized.assign(SplitVal=df_normalized[value_col].str.split(','))

    # 4. "ระเบิด" (explode) list ใน 'SplitVal' ออกเป็นแถวใหม่
    # (นี่คือหัวใจหลัก: แถวที่มี ["A", "B"] จะกลายเป็นสองแถว: แถว A และ แถว B)
    df_normalized = df_normalized.explode('SplitVal')

    # 5. ลบช่องว่าง (whitespace) ที่หัวและท้ายของค่าที่ถูก split (เช่น " Action" -> "Action")
    df_normalized['SplitVal'] = df_normalized['SplitVal'].str.strip()

    # 6. เลือกเฉพาะคอลัมน์ id และค่าที่ split แล้ว
    df_normalized = df_normalized[[id_col, 'SplitVal']]

    # 7. ลบแถวที่ซ้ำกัน (เช่น หนังเรื่องเดียวกัน, ผู้กำกับคนเดียวกัน)
    df_normalized = df_normalized.drop_duplicates()

    # 8. เปลี่ยนชื่อคอลัมน์ให้สื่อความหมาย
    final_col_name = value_col.lower().replace(" ", "_") # เช่น 'Director' -> 'director'
    df_normalized = df_normalized.rename(columns={
        id_col: 'title_cleaned',
        'SplitVal': final_col_name
    })

    return df_normalized

# --- ส่วนที่ 2:  (Main Execution) ---

try:
    # --- 2.1 กำหนดชื่อไฟล์ (ตาม Path ใน Google Drive ของคุณ) ---
    # imdb_file_path = '/content/drive/MyDrive/Python&DataViz2025/final_exam/imdb_top_1000.csv'
    # netflix_file_path = '/content/drive/MyDrive/Python&DataViz2025/final_exam/Netflix-movies.xlsx'

    df_nf_film = pd.read_excel("/content/drive/MyDrive/Final DataViz2025/Netflix-movies.xlsx",sheet_name='Netflix Films')
    # df_nf_scores = pd.read_excel("/content/drive/MyDrive/Final DataViz2025/Netflix-movies.xlsx",sheet_name='Rotten Tomatoes Score')
    df_im = pd.read_csv("/content/drive/MyDrive/Final DataViz2025/imdb_top_1000.csv")

    # --- 2.2 โหลดข้อมูล ---
    # print(f"กำลังโหลด {imdb_file_path}...")
    imdb_data = df_im

    # === [จุดแก้ไข] ===
    # ใช้ pd.read_excel และระบุ sheet_name ที่ถูกต้อง
    # print(f"กำลังโหลด {netflix_file_path} (ชีต 'Netflix Films')...")
    netflix_films_data = df_nf_film
    # ==================

    # --- 2.3 ทำความสะอาดชื่อหนัง (สร้าง 'Movie Title Cleaned') ---
    print("กำลังทำความสะอาดชื่อหนัง...")
    imdb_data['Movie Title Cleaned'] = imdb_data['Series_Title'].apply(clean_title)
    netflix_films_data['Movie Title Cleaned'] = netflix_films_data['Movie Title'].apply(clean_title)

    # --- 2.4 Normalize ตาราง Director ---
    print("กำลังประมวลผล Directors...")
    imdb_directors = normalize_column(imdb_data, 'Movie Title Cleaned', 'Director')
    netflix_directors = normalize_column(netflix_films_data, 'Movie Title Cleaned', 'Director')

    # รวมตาราง Director จาก 2 แหล่ง และลบแถวซ้ำ
    all_directors = pd.concat([imdb_directors, netflix_directors]).drop_duplicates().reset_index(drop=True)

    # --- 2.5 Normalize ตาราง Genre ---
    print("กำลังประมวลผล Genres...")
    imdb_genres = normalize_column(imdb_data, 'Movie Title Cleaned', 'Genre')
    netflix_genres = normalize_column(netflix_films_data, 'Movie Title Cleaned', 'Genre')

    # รวมตาราง Genre จาก 2 แหล่ง และลบแถวซ้ำ
    all_genres = pd.concat([imdb_genres, netflix_genres]).drop_duplicates().reset_index(drop=True)

    # --- 2.6 บันทึกไฟล์และแสดงผลลัพธ์ ---

    # บันทึกเป็น CSV (ใน /content/ ซึ่งเป็นพื้นที่ชั่วคราวของ Colab)
    output_director_path = "/content/normalized_directors.csv"
    output_genre_path = "/content/normalized_genres.csv"

    all_directors.to_csv(output_director_path, index=False)
    all_genres.to_csv(output_genre_path, index=False)

    print("\n" + "="*40)
    print("--- ตาราง Normalized Directors (ตัวอย่าง 50 แถว) ---")
    print(all_directors.head(50))

    print("\n--- ตาราง Normalized Genres (ตัวอย่าง 50 แถว) ---")
    print(all_genres.head(50))

    print(f"\n[สำเร็จ!] สร้างไฟล์ '{output_director_path}' ({len(all_directors)} แถว) และ '{output_genre_path}' ({len(all_genres)} แถว) เรียบร้อยแล้ว")

except KeyError as e:
    # กรณีที่ชื่อคอลัมน์ (เช่น 'Director' หรือ 'Genre') ไม่ถูกต้อง
    print(f"\n[เกิดข้อผิดพลาด: KeyError]")
    print(f"ไม่พบคอลัมน์ที่ระบุ: {e}")
    print("กรุณาตรวจสอบชื่อคอลัมน์ในไฟล์ของคุณ")

except FileNotFoundError as e:
    # กรณีที่หาไฟล์ไม่เจอ
    print(f"\n[เกิดข้อผิดพลาด: FileNotFoundError]")
    print(f"ไม่พบไฟล์: {e.filename}")
    print("กรุณาตรวจสอบ Path ไปยังไฟล์ใน Google Drive ของคุณ")

except Exception as e:
    # ดักจับข้อผิดพลาดอื่นๆ ที่ไม่คาดคิด
    print(f"\n[เกิดข้อผิดพลาดที่ไม่คาดคิด]: {e}")
    import traceback
    print(traceback.format_exc())

In [ ]:
all_genres

In [ ]:
all_directors

In [ ]:
df_1

# ส่งออก google sheets

In [ ]:
# ============================================================
# 🚀 Upload large DataFrames to separate Google Sheets (downsize + batch)
# ============================================================

!pip install gspread gspread_dataframe --quiet
from google.colab import auth
auth.authenticate_user()

import gspread
import pandas as pd
from google.auth import default
from gspread_dataframe import set_with_dataframe
import math

# ============================================================


In [ ]:
# ✅ เชื่อมต่อ Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# ============================================================
# ✅ กำหนดชื่อไฟล์และ DataFrame ที่จะอัปโหลด
sheets_info = {
    "df_1": df_1,
    "all_directors": all_directors,
    "all_genres": all_genres
    }

# ============================================================
# ✅ ตั้งค่า batch size (จำนวนแถวต่อครั้ง)
BATCH_SIZE = 50000

# ============================================================
# ฟังก์ชันลดขนาด DataFrame
def downsize_df(df):
    # ลด int64 → int32
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype('int32')
    # ลด float64 → float32
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    # แปลง datetime → string
    for col in df.select_dtypes(include=['datetime', 'datetimetz']).columns:
        df[col] = df[col].astype(str)
    return df

# ============================================================
# อัปโหลดทีละ DataFrame
for sheet_title, df in sheets_info.items():
    print(f"\n📤 กำลังอัปโหลด '{sheet_title}'...")

    df = downsize_df(df)  # ลดขนาดข้อมูล
    total_rows = df.shape[0]
    num_batches = math.ceil(total_rows / BATCH_SIZE)

    # สร้าง Google Sheet ใหม่
    sh = gc.create(sheet_title)
    sh.share(None, perm_type='anyone', role='writer')  # ใครก็เข้าดูได้
    ws = sh.sheet1

    print(f"ℹ️ Total rows: {total_rows}, will upload in {num_batches} batches of {BATCH_SIZE}")

    # อัปโหลด batch แรก พร้อม header
    batch = df.iloc[0:min(BATCH_SIZE, total_rows)]
    set_with_dataframe(ws, batch, include_index=False, include_column_header=True)

    # อัปโหลด batch ถัดไป (ถ้ามี)
    for i in range(1, num_batches):
        start_row = i * BATCH_SIZE
        end_row = min((i+1)*BATCH_SIZE, total_rows)
        batch = df.iloc[start_row:end_row]
        set_with_dataframe(ws, batch, include_index=False, include_column_header=False, row=start_row+1)
        print(f"⏳ Uploaded rows {start_row} to {end_row}")

    print(f"✅ Upload finished: '{sheet_title}' ({df.shape[0]} rows, {df.shape[1]} cols)")
    print(f"🔗 Google Sheets URL: https://docs.google.com/spreadsheets/d/{sh.id}")

print("\n🎉 อัปโหลดครบทุกชุดข้อมูลเรียบร้อยแล้ว!")


#Looker studio
https://lookerstudio.google.com/reporting/480afa64-d1d1-45d2-963b-cf583065f708